In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from io import StringIO
from IPython.display import HTML
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [ ]:
# Function definitions
def readESC(sounding_path, interpnan=True, handle=False):
    """
    Reads in a sounding in ESC format from a provided file path or handle (can be a StringIO object or an open
    file handle)
    """
    col_names = ['pressure','temperature','dewpoint','u_wind','v_wind','speed','direction','height',
                 'Qp_code','Qt_code','Qrh_code','Qu_code','Qv_code']
    # First read the file and extract the field widths from the 14th header line
    if not handle:
        f = open(sounding_path, 'r')
    else:
        f = sounding_path

    # Read in the header and extract some metadata from it
    dummy = f.readline()
    dummy = f.readline()
    header2 = f.readline().strip().split(':')
    # Read next header line and extract station id and wmo number from it (if it exists)
    staid_wmo_str = header2[1]
    if ' / ' in staid_wmo_str:
        staid_wmo = staid_wmo_str.strip().split(' / ')
        staid = staid_wmo[0][1:4]
        wmo = int(staid_wmo[1])
    else:
        if '. ' in staid_wmo_str:
            staid = staid_wmo_str.replace('. ', '').strip()[:4]
        else:
            staid = staid_wmo_str.strip()[:4]
            staid = staid.replace(" ", "")
        wmo = 99999
    print(staid)
    # Read the next header line and extract the location information from it
    header3 = f.readline().strip().split(':')
    location = header3[1].strip().split(',')
    print(location)
    lon = np.float(location[2])
    lat = np.float(location[3])
    elev = np.float(location[4])
    # Read the next header line and extract the time information from it
    header4 = f.readline().strip()[31:].lstrip()   
    sounding_datetime = datetime.strptime(header4, '%Y, %m, %d, %H:%M:%S')
    
    # Now read and dump the rest of the header
    for i in range(9):
        f.readline()
    
    # Except for the last header line, which is used to determine the widths of the fields
    line = f.readline().strip().split()
    fw = [len(field)+1 for field in line]

    # Now read the file into the dataframe, using the extracted field widths
    df = pd.read_fwf(f, usecols=[1, 2, 3, 5, 6, 7, 8, 14, 15, 16, 17, 18, 19],
                     names=col_names, na_values=['99999.0', '9999.0', '999.0'], widths=fw)
    
    # For some reason, need to convert all the columns to floating point here, as some get interpreted as strings
    # when they shouldn't be...
    # print(df['pressure'], df['temperature'])
    for column in df.columns:
        df[column] = df[column].astype(np.float)
    
    # Drop rows where height or pressure is NaN. TODO: Can't remember why I have to use reset_index(drop=True). 
    # Figure this out.
    df = df.dropna(subset=('height', 'pressure')).reset_index(drop=True)
    # Set the height as the index so we can use it as weights to interpolate other columns across NaN
    df = df.set_index('height')
    df['height'] = df.index
    
    if interpnan:
        # First convert direction and speed to u, v components
        df['u'], df['v'] = mpcalc.wind_components(df['speed'].values*units('m/s'),
                                                      df['direction'].values*units.degrees)
        # Now interpolate
        df = df.interpolate(method='values')
        # Finally recompute direction and speed from u, v components
        df['speed'] = mpcalc.wind_speed(df['u'].values*units('m/s'), df['v'].values*units('m/s'))
        df['direction'] = mpcalc.wind_direction(df['u'].values*units('m/s'), df['v'].values*units('m/s'))
    else:
        # Drop any rows with all NaN values for T, Td, winds
        df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed',
                               'u_wind', 'v_wind'), how='all').reset_index(drop=True)
    
    df = df[(df.Qp_code == 1.0) & (df.Qt_code == 1.0) & (df.Qrh_code == 1.0) & (df.Qu_code == 1.0) & 
            (df.Qv_code == 1.0)]

    nlines = df.count()['pressure']
    
    if not handle:
        f.close()
    
    snd_metadata = {
        'sounding_datetime': sounding_datetime,
        'lat': lat,
        'lon': lon,
        'selev': elev,
        'staid': staid,
        'wmo': wmo,
        'nlevs': nlines,
        'staid_long': staid_wmo_str
    }
    
    return snd_metadata, df


def readsharppy(path):
    """Reads in a sounding in sharppy format"""
        ## read in the file
    f = open(path, 'r')
    lines = f.read()
    data = np.array([l.strip() for l in lines.split('\n')])
    f.close()

    ## necessary index points
    title_idx = np.where( data == '%TITLE%')[0][0]
    start_idx = np.where( data == '%RAW%' )[0][0] + 1
    finish_idx = np.where( data == '%END%')[0][0]
    
    ## create the plot title
    data_header = data[title_idx + 1].split()
    location = data_header[0]
    time = data_header[1][:11]

    ## put it all together for StringIO
    full_data = '\n'.join(data[start_idx : finish_idx][:])
    sound_data = StringIO( full_data )

    ## read the data into arrays
    p, h, T, Td, wdir, wspd = np.genfromtxt( sound_data, delimiter=',', comments="%", unpack=True )
    
    col_names = ['pressure','height','temperature','dewpoint','speed','direction']
    data_dict = {key:value for (key,value) in zip(col_names,[p,h,T,Td,wspd,wdir])}
    
    df = pd.DataFrame.from_dict(data_dict)
    
    return df


def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)


def rain_Brandes(D):
    """Given a range of diameters D, compute rain fall speed curve, a quartic polynomial
       fit after Brandes et al. (2002)."""
    
    D_mm=D*1000. # get it to (mm)
    
    Vtr = -0.1021 + 4.932*D_mm - 0.9551*D_mm**2. + 0.07934*D_mm**3. - 0.002362*D_mm**4.
    
    return Vtr


def cal_xf_tf(usm, vsm, vt, H, perturb_vt=False, sigma=0.1):
    """Computes final horizontal position and residence time (relative to starting position) of a raindrop
       falling through a horizontally homogeneous layer H with terminal velocity vt and 
       storm releative mean wind given by (usm, vsm)."""
    
    if perturb_vt:
        rng = np.random.default_rng()
        vt_perts = sigma * rng.standard_normal(vt.size)
        vt = vt + vt_perts
    
    tf = H / vt
    xf = tf * usm
    yf = tf * vsm
    
    return xf, yf, tf


def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val

def interpolate_all(gridded_radar, tinterp_intv, base_field_name='reflectivity_masked'):
    # Get list of intervals in seconds between subsequent radar times
    tdiffs = gridded_radar['time_seconds'].diff(dim='time')
    
    # This list will hold all the time-interpolated grids (xarray Datasets). 
    # Can later be concatenated into a new xarray Dataset containing all times
    gridded_radar_interp_list = []
    
    # Grab first time from full dataset and restore singular time dimension
    first_time_ds = gridded_radar.isel(time=0)
    first_time_ds = first_time_ds.expand_dims(dim='time')

    gridded_radar_interp_list.append(first_time_ds)
    
#     tbgn = first_time_ds.coords['time_seconds'].values.item()  # Need to get scalar value, not 0-d
#                                                                # numpy array
    
    # Loop through the gridded_radar times, perform advection correction/interpolation between successive times
    # and add each to the list, making sure the time coordinate is consistent
    # new_time = tbgn
    for i, tdiff in enumerate(tdiffs.values):
        gridded_radar_interp_sublist = advection_correction_ds(gridded_radar.isel(time=slice(i, i+2)), 
                                                               tdiff, tinterp_intv, 
                                                               base_field_name=base_field_name)
        for t, gridded_radar_interp in enumerate(gridded_radar_interp_sublist):
#             new_time = new_time + tinterp_intv
#             new_ds = first_time_ds.copy()
#             new_ds[:] = gridded_radar_interp
#             new_ds.coords['time'] = new_ds['time'] + np.timedelta64(int(new_time), 's')
#             new_ds.coords['time_seconds'] = new_time
            gridded_radar_interp_list.append(gridded_radar_interp)
    
    return gridded_radar_interp_list


def advection_correction_ds(radar_ds, tintv_obs, tintv, base_field_name='reflectivity_masked', method="LK"):
    # Evaluate advection
    oflow_method = motion.get_method(method)
    fd_kwargs = {"buffer_mask": 10}  # avoid edge effects

    base_field = radar_ds[base_field_name]
    oflow_field = oflow_method(base_field, fd_kwargs=fd_kwargs)
    
    # Perform temporal interpolation on all variables in Dataset using the flow field derived from the "base"
    # field (by default, reflectivity)
    
    tbgn = base_field[0].coords['time_seconds'].values.item()   # Need to get scalar value, not 0-d
                                                                # numpy array
    
    radar_ds_list = []
    x, y = np.meshgrid(
        np.arange(base_field[0].shape[1], dtype=float), np.arange(base_field[0].shape[0], dtype=float),
    )
    
    new_time = tbgn
    for i in np.arange(tintv, tintv_obs + tintv, tintv):

        new_time = new_time + tintv
        
        pos1 = (y - i / tintv_obs * oflow_field[1], x - i / tintv_obs * oflow_field[0])
        pos2 = (y + (tintv_obs - i) / tintv_obs * oflow_field[1], 
                x + (tintv_obs - i) / tintv_obs * oflow_field[0])
        
        field_interp_list = []
        for field_name, field_da in radar_ds.items():
            fieldt1 = map_coordinates(field_da[0], pos1, order=1)
            fieldt2 = map_coordinates(field_da[1], pos2, order=1)
       
            field_interp = field_da.isel(time=[0]).copy()
            field_interp[:] = ((tintv_obs - i) * fieldt1 + i * fieldt2) / tintv_obs
            field_interp.coords['time'] = field_interp['time'] + np.timedelta64(int(new_time - tbgn), 's')
            field_interp.coords['time_seconds'] = new_time
            field_interp_list.append(field_interp)
        
        radar_ds_interp = xr.merge(field_interp_list)
        radar_ds_list.append(radar_ds_interp)
        
    return radar_ds_list


def advection_correction(arr, tintv_obs, tintv):
    """
    R = np.array([qpe_previous, qpe_current])
    T = time between two observations (5 min)
    t = interpolation timestep (1 min)
    """

    # Evaluate advection
    oflow_method = motion.get_method("LK")
    fd_kwargs = {"buffer_mask": 10}  # avoid edge effects
    V = oflow_method(arr, fd_kwargs=fd_kwargs)

    # Perform temporal interpolation
    # arr_d = np.zeros((arr[0].shape))
    arr_list = []
    x, y = np.meshgrid(
        np.arange(arr[0].shape[1], dtype=float), np.arange(arr[0].shape[0], dtype=float),
    )
    for i in np.arange(tintv, tintv_obs + tintv, tintv):

        pos1 = (y - i / tintv_obs * V[1], x - i / tintv_obs * V[0])
        R1 = map_coordinates(arr[0], pos1, order=1)
        
        pos2 = (y + (tintv_obs - i) / tintv_obs * V[1], x + (tintv_obs - i) / tintv_obs * V[0])
        R2 = map_coordinates(arr[1], pos2, order=1)

        arr_interp = ((tintv_obs - i) * R1 + i * R2) / tintv_obs
        arr_list.append(arr_interp)

    return arr_list


def plot_animation(xplt, yplt, field, clevels, cbarlabel=None, cbarintv=None, cmap='pyart_HomeyerRainbow', 
                   norm=None, PIPS_list=None, PIPS_xy_list=None, ax=None, ptype='pcolor', axestickintv=10000., 
                   axeslimits=None):
    
    if norm is None:
        norm = cm.colors.Normalize(vmin=clevels[0], vmax=clevels[-1])
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    else:
        fig = ax.get_figure()
    ims = []
    for i, var in enumerate(var_da):
        plotdata = []
        time = np.datetime_as_string(var.coords['time'].values, unit='m')  # Ugly, but whatever
        
        title = ax.text(0.5,1.05,"Time: {}".format(time), 
                        size=plt.rcParams["axes.titlesize"],
                        ha="center", transform=ax.transAxes)
        plotdata.append(title)
        
        if ptype == 'pcolor':
            ci = ax.pcolormesh(xplt, yplt, var.squeeze(), vmin=clevels[0], vmax=clevels[-1], cmap=cmap, 
                                     norm=norm)
            plotdata.append(ci)
        else:
            ci = ax.contourf(xplt, yplt, var.squeeze(), levels=clevels, 
                             cmap=cmap, norm=norm)
            plotdata.extend(ci.collections)
            
        if PIPS_list is not None and PIPS_xy_list is not None:
            # Plot PIPS locations
            for PIPS, PIPS_xy in zip(PIPS_list, PIPS_xy_list):
                PIPS_x = PIPS_xy[0]
                PIPS_y = PIPS_xy[1]
                ax.plot([PIPS_x], [PIPS_y], 'k*')
        if i == 0.:
            if cbarintv is None:
                cbarintv = clevels[1] - clevels[0]
            cbarlevels = ticker.MultipleLocator(base=cbarintv)
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(ci, orientation='vertical', ticks=cbarlevels, cax=cax)
            if cbarlabel is not None:
                cax.set_ylabel(cbarlabel)
            formatter = ticker.FuncFormatter(mtokm)
            ax.xaxis.set_major_formatter(formatter)
            ax.yaxis.set_major_formatter(formatter)
            ax.xaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
            ax.yaxis.set_major_locator(ticker.MultipleLocator(base=axestickintv))
            ax.set_xlabel('km')
            ax.set_ylabel('km')
            if axeslimits is None:
                xmin = xplt[0]
                xmax = xplt[-1]
                ymin = yplt[0]
                ymax = yplt[-1]
            else:
                xmin, xmax, ymin, ymax = axeslimits
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.set_aspect('equal')
            
        ims.append(plotdata)
    
    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                    repeat_delay=1000)
    plt.close()
    return ani


def interp_pyart_grid_to_PIPS(grid_ds, PIPS_loc_list):
    """Interpolate pyart gridded radar to PIPS locations given an xarray dataset with pyart grid info and a list
       of PIPS lats, lons, and altitudes. Returns lists of PIPS x, y, and z locations in radar grid coordinates
       and a list of xarray Datasets for each grid variable interpolated to each PIPS location."""

    # TODO: instead of returning lists, maybe make the PIPS name a dimension of a new xarray Dataset
    
    ctrlat = np.float(grid_ds.origin_latitude)
    ctrlon = np.float(grid_ds.origin_longitude)
    print(ctrlat, ctrlon)

    radar_at_PIPS_list = []
    PIPS_x_list = []
    PIPS_y_list = []
    PIPS_z_list = []

    for PIPS_loc in PIPS_locs:
        PIPS_lat = PIPS_loc[0]
        PIPS_lon = PIPS_loc[1]
        PIPS_alt = PIPS_loc[2]
        radar_alt = np.float(grid_ds.origin_altitude)
        
        PIPS_z = PIPS_alt - radar_alt
        PIPS_z_list.append(PIPS_z)
        # Use this function to get the x and y coords of the PIPS. Note that this will only be correct 
        # if the radar
        # grid was created using the default pyart aeqd projection.
        PIPS_x, PIPS_y = pyart.core.geographic_to_cartesian_aeqd(PIPS_lon, PIPS_lat, ctrlon, ctrlat)
        PIPS_x = PIPS_x.squeeze().item()
        PIPS_y = PIPS_y.squeeze().item()
        PIPS_x_list.append(PIPS_x)
        PIPS_y_list.append(PIPS_y)
        print('PIPS lat, lon, alt: ', PIPS_lat, PIPS_lon, PIPS_alt)
        print('PIPS x, y, z: ', PIPS_x, PIPS_y, PIPS_z)
        radar_at_PIPS_ds = grid_ds.interp(x=PIPS_x, y=PIPS_y)
        radar_at_PIPS_list.append(radar_at_PIPS_ds)

    return PIPS_x_list, PIPS_y_list, PIPS_z_list, radar_at_PIPS_list


def write_cm1(sounding_path, p_sfc, theta_sfc, qv_sfc, z_snd, theta_snd, qv_snd, u_snd, v_snd):
    '''Writes a CM1/COMMAS/WRF format sounding file given the appropriate arrays.
       IMPORTANT: Following https://www2.mmm.ucar.edu/people/bryan/cm1/soundings/,
       units should be in m for height, hPa for pressure, K for potential temperature, g/kg for 
       water vapor mixing ratio, and m/s for the u and v wind components. Will use metpy/pint to convert to 
       these units just to be safe
    '''
    print()
    print('********************')
    print()
    print('creating CM1 sounding...')
    
    # Convert units to that needed for cm1/wrf/commas format (will do nothing, of course, if the units are 
    # already correct)
    z_snd = z_snd.to(units('m'))
    p_sfc = p_sfc.to(units('hPa'))
    theta_sfc = theta_sfc.to(units('K'))
    theta_snd = theta_snd.to(units('K'))
    qv_sfc = qv_sfc.to(units('g/kg'))
    qv_snd = qv_snd.to(units('g/kg'))
    u_snd = u_snd.to(units('m/s'))
    v_snd = v_snd.to(units('m/s'))
    
    # Construct the header
    header = f'{p_sfc.m:8.8f}    {theta_sfc.m:8.8f}   {qv_sfc.m:8.8f}'

    # Stuff the sounding arrays into a single 2D array to prep for the call to np.savetxt()
    sounding_arr = np.transpose((z_snd.m, theta_snd.m, qv_snd.m, u_snd.m, v_snd.m))

    # Write the sounding to the file
    # GOTCHA: have to put in "comments=''" otherwise savetxt prepends a "#" to the header line...
    np.savetxt(sounding_path, sounding_arr, fmt='%8.8f', header=header, comments='')


In [ ]:
# Read in the file containing the original gridded radar at the top of the sorting layer

# 03/25 case
# radar_name = 'KGWX'
# radar_type= 'NEXRAD'
# date = '0325'
# radar_start_datetimestamp = '20170325170500'
# radar_end_datetimestamp = '20170325183559'
# height = 2000.

# For 03/30/22 case (PERiLS-2022 IOP2)
radar_name = 'KGWX'
radar_type= 'NEXRAD'
date = '0330'
radar_start_datetimestamp = '20220330220424'
radar_end_datetimestamp = '20220331035633'
height = 500.

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

radar_basedir = \
    '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/GRID/IOP2/KGWX'
gridded_radar_dir = os.path.join(radar_basedir, 'combined')
plot_dir = os.path.join(gridded_radar_dir, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

radar_start_timestamp = datetime_start.strftime('%Y%m%d%H%M')
radar_end_timestamp = datetime_end.strftime('%Y%m%d%H%M')

# Read in original gridded radar at top of sorting layer
gridded_radar_interp_filename = '{}_{}_{}_z{:d}_gridded_interp_retr.nc'.format(radar_name, radar_start_timestamp,
                                                                          radar_end_timestamp, int(height))
gridded_radar_interp_filepath = os.path.join(gridded_radar_dir, gridded_radar_interp_filename)
gridded_radar_interp_ds = xr.open_dataset(gridded_radar_interp_filepath)

In [ ]:
# Read in PIPS data (just to get lat/lon for now)
# deployment = 'IOP1A_D1_2017'
# PIPS_list = ['PIPS1A', 'PIPS1B', 'PIPS2B']
# PIPS_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15'

deployment = 'IOP2_033022'
PIPS_list = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
PIPS_data_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/IOP2_033022/netcdf'


PIPS_ds_list = []
PIPS_locs = []

for PIPS in PIPS_list:
    PIPS_filename = 'parsivel_combined_{}_{}_60s.nc'.format(deployment, PIPS)
    PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
    PIPS_ds = xr.load_dataset(PIPS_filepath)
    PIPS_ds_list.append(PIPS_ds)
    PIPS_loc = eval(PIPS_ds.location)
    PIPS_locs.append(PIPS_loc)

In [ ]:
# Find PIPS x, y location by interpolating to its lat/lon point
PIPS_x_list, PIPS_y_list, PIPS_z_list, radar_at_PIPS_ds_list = interp_pyart_grid_to_PIPS(gridded_radar_interp_ds, 
                                                                                         PIPS_locs)
    
PIPS_z_mean = np.array(PIPS_z_list).mean()
print(PIPS_z_list)
print(PIPS_z_mean)
PIPS_x_arr = np.array(PIPS_x_list)
PIPS_y_arr = np.array(PIPS_y_list)

print(PIPS_x_arr, PIPS_y_arr)

PIPS_xy_list = [(PIPS_x, PIPS_y) for PIPS_x, PIPS_y in zip(PIPS_x_list, PIPS_y_list)]

In [ ]:
# Extract within a small bounding box and for a limited time slice

# For 04/30 case
# tstart = '2017-04-30T20:00'
# tstop = '2017-04-30T21:30'

# For 03/27 case
# tstart = '2017-03-27T19:30'
# tstop = '2017-03-27T21:00'

# For 03/25 case
# tstart = '2017-03-25T17:05'
# tstop = '2017-03-25T18:35'

# For PERiLS IOP2 2022 case
tstart = '2022-03-30T23:30'
tstop = '2022-03-31T01:15'

# lat_bgn = 33.0
# lat_end = 34.5
# lon_bgn = -89.0
# lon_end = -88.0

# Instead of lat and lon bounds, use projection x and y coordinates and center on the mean location
# of the PIPS +/- some range

PIPS_x_mean = np.mean(PIPS_x_arr)
PIPS_y_mean = np.mean(PIPS_y_arr)

print(PIPS_x_mean, PIPS_y_mean)

x_hw = 25000.
y_hw = 25000.

xbgn = PIPS_x_mean - x_hw
xend = PIPS_x_mean + x_hw
ybgn = PIPS_y_mean - y_hw
yend = PIPS_y_mean + y_hw

# ibgn = 50
# iend = 150
# jbgn = 125
# jend = 225
# level = 2
# z_level = gridded_radar.point_z['data'][level, 0, 0]
# z_level = 1000.

# This old way of doing it was based on a lie. The pyart function grid.to_xarray() was erroneously outputting
# the lat and lon coordinates as 1D coordinates dimensioned by y and x, respectively. In reality they should
# each be dimensioned by (y, x). This has been fixed in the latest version of pyart.
# gridded_radar_interp_ds = gridded_radar_interp_ds.swap_dims({'y': 'lat', 'x': 'lon'})
# gridded_radar_subgrid = gridded_radar_interp_ds.sel(lat=slice(lat_bgn, lat_end), lon=slice(lon_bgn, lon_end),
#                                                     time=slice(tstart, tstop))
gridded_radar_subgrid = gridded_radar_interp_ds.sel(x=slice(xbgn, xend), y=slice(ybgn, yend), 
                                                    time=slice(tstart, tstop))

# gridded_radar_subgrid = gridded_radar_subgrid.squeeze()
# gridded_radar_subgrid = gridded_radar_subgrid.transpose('time', 'lat', 'lon')
print(gridded_radar_subgrid)

In [ ]:
gridded_radar_subgrid

In [ ]:
# Read in sounding file to get low-level wind field and then derive storm-relative wind
# Storm motion taken from subjective reflectivity tag tracking using GRLevel2
# EDIT: don't need storm motion because it is implicitly handled in time-dependent trajectory model
# ustorm = 12.51
# vstorm = 12.95

# EDIT: setting ustorm, vstorm to 0 to force ground-relative flow
ustorm = 0.
vstorm = 0.

# sounding_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/2017/soundings'
# sounding_filename = 'Courtland_1759.txt'
# # sounding_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2017/soundings/COMP5mb'
# # sounding_filename = 'Hollywood_201704301954.cls'
# sounding_path = os.path.join(sounding_dir, sounding_filename)
# sounding_metadata, sounding_df = readESC(sounding_path)

# For PERiLS IOP2 UIUC SONDE4
sounding_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/PERiLS/obsdata/2022/non-radar_QC_Illinois/20220330_IOP02/SONDE4/sounding/L2'
sounding_filename = 'SPC_20220330_SONDE4_2328.txt'
sounding_path = os.path.join(sounding_dir, sounding_filename)

sounding_df = readsharppy(sounding_path)
wind_dir = sounding_df['direction'].values*units.degrees
# print(wind_dir)
wind_speed_kts = sounding_df['speed'].values*units.knots 
wind_speed_ms = wind_speed_kts.to(units('m/s'))

# print(wind_speed_ms)
u, v = wind_components(wind_speed_ms, wind_dir)

sounding_df['u'] = u
sounding_df['v'] = v

In [ ]:
sounding_df

In [ ]:
# Get the air density from the sounding at the chosen height
sounding_df_one_height = sounding_df.loc[sounding_df['height'] == height]
T = sounding_df_one_height['temperature'].values * units['degC']
Td = sounding_df_one_height['dewpoint'].values * units['degC']
p = sounding_df_one_height['pressure'].values * units['hPa']

RH = mpcalc.relative_humidity_from_dewpoint(T, Td)
qv = mpcalc.mixing_ratio_from_relative_humidity(p, T, RH)
rhoa = mpcalc.density(p, T, qv)

print(rhoa)

In [ ]:
# Dump sounding to CM1/COMMAS format
cm1_sounding_dir = '/Users/dawson29/Projects/commas_sed_tests/PERiLS_IOP2_2022/'
cm1_sounding_filename = 'SPC_20220330_SONDE4_2328.commas'
cm1_sounding_path = os.path.join(cm1_sounding_dir, cm1_sounding_filename)

z_snd = sounding_df['height']
z_AGL_snd = z_snd - z_snd[0]
z_AGL_snd = z_AGL_snd.values * units('m')
p_snd = sounding_df['pressure'].values * units('hPa')
p_sfc = p_snd[0]
T_snd = sounding_df['temperature'].values * units('degC')
Td_snd = sounding_df['dewpoint'].values * units('degC')

theta_snd = mpcalc.potential_temperature(p_snd, T_snd)
theta_sfc = theta_snd[0]

RH_snd = mpcalc.relative_humidity_from_dewpoint(T_snd, Td_snd)
qv_snd = mpcalc.mixing_ratio_from_relative_humidity(p_snd, T_snd, RH_snd)
qv_sfc = qv_snd[0]

u_snd = sounding_df['u'].values * units('m/s')
v_snd = sounding_df['v'].values * units('m/s')

write_cm1(cm1_sounding_path, p_sfc, theta_sfc, qv_sfc, z_AGL_snd, theta_snd, qv_snd, u_snd, v_snd)

In [ ]:
# Compute the q, Nt, and Z moments from the gamma distribution parameters

retr_suffix = 'Z01_4dB'

lamda = gridded_radar_subgrid[f'lamda_{retr_suffix}'] * 1000. # get to m^-1
alpha = gridded_radar_subgrid[f'mu_{retr_suffix}']
N0 = gridded_radar_subgrid[f'N0_{retr_suffix}'] * 1000**(1 + alpha) # get to m^-4

# Already have Nt
Ntr = gridded_radar_subgrid[f'Nt_{retr_suffix}']
qr = dsd.calc_qr_gamma(rhoa.m, N0, lamda, alpha)
Zr = dsd.calc_Zr_lin_gamma(rhoa.m, qr, Ntr, alpha)


In [ ]:
qr

In [ ]:
# Pick a short range of times for testing:

tbgn = '2022-03-31T00:00:00'
tend = '2022-03-31T00:30:00'

qr_tslice = qr.sel(time=slice(tbgn, tend)).fillna(0.0)
Ntr_tslice = Ntr.sel(time=slice(tbgn, tend)).fillna(0.0)
Zr_tslice = Zr.sel(time=slice(tbgn, tend)).fillna(0.0)

In [ ]:
qr_tslice

In [ ]:
qr_tslice.isel(time=0).plot()

In [ ]:
for qr_onetime in qr_tslice:
    # timestamp = np.datetime_as_string(qr_onetime.coords['time'].values, unit='m')
    time = qr_onetime['time'].values
    # This is insane. Why can't numpy datetime64 and python datetime objects play nice with each other?
    time_dt = time.astype('datetime64[us]').astype(datetime)
    timestamp = time_dt.strftime('%Y%m%d%H%M%S')
    print(timestamp)

In [ ]:
# Now dump to unformatted fortran binary files (one per time) for reading into COMMAS
from scipy.io import FortranFile

output_dir = '/Users/dawson29/Projects/commas_sed_tests/PERiLS_IOP2_2022/top_boundary_rain'

# Loop through the times, construct the file name using the time in seconds since the start, and dump to 
# unformatted binary

timesecint_start = int(qr_tslice['time_seconds'][0].values)
print(timesecint_start)

for qr_onetime, Ntr_onetime, Zr_onetime in zip(qr_tslice, Ntr_tslice, Zr_tslice):
    timesecint = int(qr_onetime['time_seconds'].values) - timesecint_start
    output_filename = f'test_rain_top_{timesecint:06d}.dat'
    output_path = os.path.join(output_dir, output_filename)
    print(f"Writing {output_path}!")
    
    uff = FortranFile(output_path, 'w')
    # Need to cast to float32 since that is the precision COMMAS is expecting
    uff.write_record(qr_onetime.squeeze().values.astype(np.float32))
    uff.write_record(Ntr_onetime.squeeze().values.astype(np.float32))
    uff.write_record(Zr_onetime.squeeze().values.astype(np.float32))
    uff.close()

In [ ]:
fr = FortranFile('test_rain_top.dat', 'r')
qr_r = fr.read_reals(float)
print(qr_r)